In [1]:
from scipy import interpolate
import matplotlib.pyplot as plt
import numpy as np 

D:\Anaconda\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\Anaconda\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
D:\Anaconda\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)

Bad key "text.kerning_factor" on line 4 in
D:\Anaconda\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
import numpy as np
import torch
from torch.utils.data.dataset import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import scipy.linalg as slin
import scipy.sparse as sp
import networkx as nx
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import glob
import re
import math
from torch.optim.adam import Adam
from utils import *
from statistics import mean


In [3]:
import math 
def cos(x):
    return math.cos(x/4*math.pi)*0.8

In [4]:
def data_create(seed,time_stamp):
  np.random.seed(seed)
  x = np.array(range(0, time_stamp))
  y = np.array([cos(i) for i in x]) #generate coefficient

  base_DAG=np.zeros((5, 5))
  base_DAG[0,4]=y[0]##edited to be coeffcient with error
  base_graph=nx.from_numpy_matrix(base_DAG,create_using=nx.DiGraph)
  X_all = simulate_lsem(base_graph,30, 'Binary', 1,noise_scale=0.1)
  for i in range(1,time_stamp):
      base_DAG[0,4]=y[i]
      base_graph=nx.from_numpy_matrix(base_DAG,create_using=nx.DiGraph)
      X = simulate_lsem(base_graph,30, 'Binary', 1,noise_scale=0.1)
      X_all=np.append(X_all,X,axis=0)
  return X_all

In [5]:
n = 30 # The number of samples of data.
d = 5 # The number of variables in data.
time_stamp=10 #no. of timestamp
np.random.seed(1234567) #Random seed
n_times=30
seed_list=np.random.randint(1, 1000000, size=n_times)
seed=seed_list[0]
X_all=data_create(seed,time_stamp) #create data

## piecewise ANOCA

In [6]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import pickle
import os
import random

import torch.optim as optim
from torch.optim import lr_scheduler
import math
from utils import *

from multiprocessing import Pool
import multiprocessing
n_cores = multiprocessing.cpu_count()
from numpy.random import randn
from random import seed as rseed
from numpy.random import seed as npseed


In [7]:
# compute constraint h(A) value
def _h_A(A, m):
    expm_A = matrix_poly(A*A, m)
    h_A = torch.trace(expm_A) - m
    return h_A

prox_plus = torch.nn.Threshold(0.,0.)

def stau(w, tau):
    w1 = prox_plus(torch.abs(w)-tau)
    return torch.sign(w)*w1


def update_optimizer(optimizer, original_lr, c_A):
    '''related LR to c_A, whenever c_A gets big, reduce LR proportionally'''
    MAX_LR = 1e-2
    MIN_LR = 1e-4

    estimated_lr = original_lr / (math.log10(c_A) + 1e-10)
    if estimated_lr > MAX_LR:
        lr = MAX_LR
    elif estimated_lr < MIN_LR:
        lr = MIN_LR
    else:
        lr = estimated_lr

    # set LR
    for parame_group in optimizer.param_groups:
        parame_group['lr'] = lr

    return optimizer, lr

#===================================
# training:
#===================================

def train(epoch, lambda_A, c_A, optimizer,old_lr):
    t = time.time()
    nll_train = []
    kl_train = []
    mse_train = []
    shd_trian = []

    encoder.train()
    decoder.train()
    scheduler.step()


    # update optimizer
    optimizer, lr = update_optimizer(optimizer, old_lr, c_A)


    for batch_idx, (data, relations) in enumerate(train_loader):

#         if args.cuda:
#             data, relations = data.cuda(), relations.cuda()
        data, relations = Variable(data).double(), Variable(relations).double()

        # reshape data
        relations = relations.unsqueeze(2)

        optimizer.zero_grad()

        enc_x, logits, origin_A, adj_A_tilt_encoder, z_gap, z_positive, myA, Wa = encoder(data, rel_rec, rel_send)  # logits is of size: [num_sims, z_dims]
        edges = logits

        dec_x, output, adj_A_tilt_decoder = decoder(data, edges,d * x_dims, rel_rec, rel_send, origin_A, adj_A_tilt_encoder, Wa)

        if torch.sum(output != output):
            print('nan error\n')

        target = data
        preds = output
        variance = 0.

        # reconstruction accuracy loss
        loss_nll = nll_gaussian(preds, target, variance)

        # KL loss
        loss_kl = kl_gaussian(logits)

        # ELBO loss:
        loss = loss_kl + loss_nll

        # add A loss
        one_adj_A = origin_A # torch.mean(adj_A_tilt_decoder, dim =0)
        sparse_loss = tau_A * torch.sum(torch.abs(one_adj_A))

        # compute h(A)
        h_A = _h_A(origin_A, d)
        loss += lambda_A * h_A + 0.5 * c_A * h_A * h_A + 100. * torch.trace(origin_A*origin_A) + sparse_loss #+  0.01 * torch.sum(variance * variance)


        loss.backward()
        loss = optimizer.step()

        myA.data = stau(myA.data, tau_A*lr)


        mse_train.append(F.mse_loss(preds, target).item())
        nll_train.append(loss_nll.item())
        kl_train.append(loss_kl.item())

    #print(h_A.item())

    return np.mean(np.mean(kl_train)  + np.mean(nll_train)), np.mean(nll_train), np.mean(mse_train), origin_A, optimizer, lr



In [8]:
n = 30 # The number of samples of data.
d = 5 # The number of variables in data.
x_dims = 1 # The number of input dimensions: default 1.
z_dims = d # The number of latent variable dimensions: default the same as variable size.
epochs = 200 # Number of epochs to train.
batch_size = 10 # Number of samples per batch. note: should be divisible by sample size, otherwise throw an error.
n_var=5

n_times=30 #no. of replicates
time_stamp=10 #no. of timestamp
np.random.seed(1234567) #Random seed
seed_list=np.random.randint(1, 1000000, size=n_times)
average_coef_list=np.zeros((n_times,time_stamp,n_var,n_var))
B_list=np.zeros((n_times,d, d))
FDR_total=[]
TPR_total=[]
SHD_total=[]
time_list=[]
for replicate in range(n_times):
  seed=seed_list[replicate]
  print(seed)
  X_all=data_create(seed,time_stamp) #create data
  average_list=np.zeros((time_stamp,d, d))
  FDR_list_piece=[]
  TPR_list_piece=[]
  SHD_list_piece=[]
  base_DAG=np.zeros((5, 5))
  ####estimate at each time_stamp####
  timestart=time.time()
  for j in range(time_stamp):
  # ----------- Configurations:
      k_max_iter = int(1e2) # The max iteration number for searching parameters.
      original_lr = 3e-3  # Initial learning rate.
      encoder_hidden = d^2 # Number of hidden units, adaptive to dimension of nodes (d^2).
      decoder_hidden = d^2 # Number of hidden units, adaptive to dimension of nodes (d^2).
      temp = 0.5 # Temperature for Gumbel softmax.
      factor = True # Factor graph model.
      encoder_dropout = 0.0 # Dropout rate (1 - keep probability).
      decoder_dropout = 0.0 # Dropout rate (1 - keep probability).
      tau_A = 0. # Coefficient for L-1 norm of matrix B.
      lambda1 = 0. # Coefficient for DAG constraint h1(B).
      c_B = 1 # Coefficient for absolute value h1(B).
      h1_tol = 1e-8 # The tolerance of error of h1(B) to zero.
      lr_decay = 200 # After how many epochs to decay LR by a factor of gamma. 
      gamma = 1.0 # LR decay factor. 
      ######################


      X=X_all[(j*30):(j*30+30),:]


      np.random.seed(seed)
      random.seed(seed)
      torch.manual_seed(seed)
      feat_train = torch.FloatTensor(X)
      feat_valid = torch.FloatTensor(X)
      feat_test = torch.FloatTensor(X)

      # Reconstruct itself
      train_data = TensorDataset(feat_train, feat_train)
      valid_data = TensorDataset(feat_valid, feat_train)
      test_data = TensorDataset(feat_test, feat_train)

      train_loader = DataLoader(train_data, batch_size = batch_size)
      valid_loader = DataLoader(valid_data, batch_size = batch_size)
      test_loader = DataLoader(test_data, batch_size = batch_size)

      # ----------- Load modules:
      off_diag = np.ones([d, d]) - np.eye(d) # Generate off-diagonal interaction graph
      rel_rec = np.array(encode_onehot(np.where(off_diag)[1]), dtype = np.float64)
      rel_send = np.array(encode_onehot(np.where(off_diag)[0]), dtype = np.float64)
      rel_rec = torch.DoubleTensor(rel_rec)
      rel_send = torch.DoubleTensor(rel_send)
      adj_A = np.zeros((d, d)) # Add adjacency matrix

      encoder = MLPEncoder(d * x_dims, x_dims, encoder_hidden,
                              int(z_dims), adj_A,
                              batch_size = batch_size,
                              do_prob = encoder_dropout, factor = factor).double()
      decoder = MLPDecoder(d * x_dims,
                              z_dims, x_dims, encoder,
                              data_variable_size = d,
                              batch_size = batch_size,
                              n_hid=decoder_hidden,
                              do_prob=decoder_dropout).double()

      # ----------- Set up optimizer:
      optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr = original_lr)
      scheduler = lr_scheduler.StepLR(optimizer, step_size = lr_decay,
                                      gamma = gamma)

      rel_rec = Variable(rel_rec)
      rel_send = Variable(rel_send)

      # ----------- Main:
      best_ELBO_loss = np.inf
      best_NLL_loss = np.inf
      best_MSE_loss = np.inf
      h1_B_new = torch.tensor(1.)
      h2_B_new = 1
      h1_B_old = np.inf
      h2_B_old = np.inf
      lr = original_lr
    

      try:
          for step_k in range(k_max_iter):
              while c_B< 1e+20:
                  for epoch in range(epochs):
                      old_lr = lr 
                      ELBO_loss, NLL_loss, MSE_loss, origin_B, optimizer, lr = train(epoch, lambda1, c_B, optimizer, old_lr)

                      if ELBO_loss < best_ELBO_loss:
                          best_ELBO_loss = ELBO_loss

                      if NLL_loss < best_NLL_loss:
                          best_NLL_loss = NLL_loss

                      if MSE_loss < best_MSE_loss:
                          best_MSE_loss = MSE_loss

                  if ELBO_loss > 2 * best_ELBO_loss:
                      break

                  # Update parameters
                  B_new = origin_B.data.clone()
                  h1_B_new =  _h_A(B_new,d)
                  if h1_B_new.item() > 0.25 * h1_B_old:
                      c_B *= 10
                  else:
                      break

              # Update parameters    
              h1_B_old = h1_B_new.item()
              lambda1 += c_B * h1_B_new.item()

              if h1_B_new.item() <= h1_tol:
                  break

      except KeyboardInterrupt:
          print('KeyboardInterrupt')

      predB = np.matrix(origin_B.data.clone().numpy())
      print('Best ELBO Loss :', best_ELBO_loss)
      print('Best NLL Loss :', best_NLL_loss)
      print('Best MSE Loss :', best_MSE_loss)
      #calculate_effect(predB)
      print(j)
      average_list[j,:,:]=predB



  average_coef_list[replicate,:,:,:]=average_list #average coef save to matrix
  np.save("cos_10_30_DAGGNN",average_coef_list)
#   timeend=time.time()
#   time_list.append(timeend-timestart)
  #print(timeend-timestart)
  #####write at every epoch
  #df.to_csv("cos_rep10.csv")
  print(replicate)

913812


D:\Anaconda\lib\site-packages\torch\optim\lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Best ELBO Loss : 0.0007491288473453828
Best NLL Loss : 4.6977461260965673e-07
Best MSE Loss : 1.8790984504386271e-07
0
Best ELBO Loss : 0.0007875961541128443
Best NLL Loss : 1.8774038345363111e-06
Best MSE Loss : 7.509615338145246e-07
1
Best ELBO Loss : 0.0010146159258639018
Best NLL Loss : 1.3500929601663986e-06
Best MSE Loss : 5.400371840665594e-07
2
Best ELBO Loss : 0.0005517687195360088
Best NLL Loss : 4.3540947553365723e-07
Best MSE Loss : 1.741637902134629e-07
3
Best ELBO Loss : 0.0006888775840709305
Best NLL Loss : 6.553838564478916e-07
Best MSE Loss : 2.6215354257915665e-07
4
Best ELBO Loss : 0.0007283755767546325
Best NLL Loss : 1.0462339911031858e-06
Best MSE Loss : 4.184935964412743e-07
5
Best ELBO Loss : 0.0009239771294719093
Best NLL Loss : 1.4039456262243658e-06
Best MSE Loss : 5.615782504897464e-07
6
Best ELBO Loss : 0.0006554161127119793
Best NLL Loss : 5.054791534808244e-07
Best MSE Loss : 2.0219166139232973e-07
7
Best ELBO Loss : 0.0010309582940772246
Best NLL Loss : 

Best ELBO Loss : 0.0012713806163541959
Best NLL Loss : 2.111809822755675e-06
Best MSE Loss : 8.4472392910227e-07
0
Best ELBO Loss : 0.0008884329927630486
Best NLL Loss : 9.010024820186033e-07
Best MSE Loss : 3.604009928074414e-07
1
Best ELBO Loss : 0.0011974220701563153
Best NLL Loss : 1.8091617368009136e-06
Best MSE Loss : 7.236646947203655e-07
2
Best ELBO Loss : 0.0008800997533264246
Best NLL Loss : 3.6946979184203137e-06
Best MSE Loss : 1.4778791673681256e-06
3
Best ELBO Loss : 0.0012697448411324037
Best NLL Loss : 1.4195036545857085e-06
Best MSE Loss : 5.678014618342833e-07
4
Best ELBO Loss : 0.000935384380180006
Best NLL Loss : 2.6083438845790495e-06
Best MSE Loss : 1.0433375538316197e-06
5
Best ELBO Loss : 0.001409973030092313
Best NLL Loss : 3.8103974142195536e-06
Best MSE Loss : 1.5241589656878213e-06
6
Best ELBO Loss : 0.0007837043159905517
Best NLL Loss : 9.422486479475058e-07
Best MSE Loss : 3.7689945917900235e-07
7
Best ELBO Loss : 0.0012763598742135483
Best NLL Loss : 2.59

Best ELBO Loss : 0.0014070731898049534
Best NLL Loss : 3.064071701479751e-06
Best MSE Loss : 1.2256286805919003e-06
0
Best ELBO Loss : 0.0008066780207358955
Best NLL Loss : 1.0253146571758433e-06
Best MSE Loss : 4.1012586287033726e-07
1
Best ELBO Loss : 0.0011204020502211343
Best NLL Loss : 2.5428166330202667e-06
Best MSE Loss : 1.0171266532081067e-06
2
Best ELBO Loss : 0.0007825344924010602
Best NLL Loss : 1.1966499778643097e-06
Best MSE Loss : 4.786599911457239e-07
3
Best ELBO Loss : 0.0007143076489448825
Best NLL Loss : 1.116213340122126e-06
Best MSE Loss : 4.464853360488504e-07
4
Best ELBO Loss : 0.000799672468746784
Best NLL Loss : 1.1867562642859378e-06
Best MSE Loss : 4.74702505714375e-07
5
Best ELBO Loss : 0.0011038500809785747
Best NLL Loss : 2.5655885754921433e-06
Best MSE Loss : 1.0262354301968574e-06
6
Best ELBO Loss : 0.0008043041608181917
Best NLL Loss : 1.327065835000311e-06
Best MSE Loss : 5.308263340001243e-07
7
Best ELBO Loss : 0.0012206977631760369
Best NLL Loss : 1.

Best ELBO Loss : 0.0008460187215494363
Best NLL Loss : 1.001239058920977e-06
Best MSE Loss : 4.0049562356839084e-07
0
Best ELBO Loss : 0.0009458592378665601
Best NLL Loss : 1.4932827362155115e-06
Best MSE Loss : 5.973130944862047e-07
1
Best ELBO Loss : 0.0011852117098283823
Best NLL Loss : 2.988911962517336e-06
Best MSE Loss : 1.1955647850069345e-06
2
Best ELBO Loss : 0.0007301344619121198
Best NLL Loss : 5.767578711536975e-07
Best MSE Loss : 2.3070314846147904e-07
3
Best ELBO Loss : 0.0008038866236662889
Best NLL Loss : 8.234837169594745e-07
Best MSE Loss : 3.2939348678378977e-07
4
Best ELBO Loss : 0.0007639677036910754
Best NLL Loss : 6.774954032018131e-07
Best MSE Loss : 2.709981612807252e-07
5
Best ELBO Loss : 0.0011580885909319642
Best NLL Loss : 3.392666852716911e-05
Best MSE Loss : 1.3570667410867643e-05
6
Best ELBO Loss : 0.0008148100769889727
Best NLL Loss : 1.540795062785798e-06
Best MSE Loss : 6.16318025114319e-07
7
Best ELBO Loss : 0.0010220480258013105
Best NLL Loss : 1.15

Best ELBO Loss : 0.0029229436182485052
Best NLL Loss : 8.467802710115797e-06
Best MSE Loss : 3.3871210840463187e-06
0
Best ELBO Loss : 0.0027755288096923095
Best NLL Loss : 5.648106204982652e-06
Best MSE Loss : 2.2592424819930607e-06
1
Best ELBO Loss : 0.002316270812157651
Best NLL Loss : 7.3750110632177846e-06
Best MSE Loss : 2.950004425287114e-06
2
Best ELBO Loss : 0.0024187336569607334
Best NLL Loss : 5.7617103399144675e-06
Best MSE Loss : 2.3046841359657864e-06
3
Best ELBO Loss : 0.0026377389714354184
Best NLL Loss : 5.895789576036343e-06
Best MSE Loss : 2.358315830414537e-06
4
Best ELBO Loss : 0.002299412923882918
Best NLL Loss : 4.676398243125061e-06
Best MSE Loss : 1.8705592972500242e-06
5
Best ELBO Loss : 0.0019410015332135815
Best NLL Loss : 5.180261301108958e-06
Best MSE Loss : 2.072104520443583e-06
6
Best ELBO Loss : 0.0019773927092359787
Best NLL Loss : 5.636038085052932e-06
Best MSE Loss : 2.254415234021173e-06
7
Best ELBO Loss : 0.0017114480508102303
Best NLL Loss : 5.122